# 第10章 使用一等函数实现设计模式

## 10.1 案例分析：重构策略模式

**典型示例：**

根据客户的属性或订单中的商品计算折扣，假如某个网店制定了以下折扣规则：
1. 有1000或以上积分的顾客，每个订单享5%折扣。
2. 同一个订单中，单个商品的数量达到20个或以上，享10%折扣。
3. 订单中不同商品的数量达到10个或以上，享7%折扣。

### 10.1.1 典型策略模式

In [1]:
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity


class Order(NamedTuple):
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'


class Promotion(ABC):
    """策略：抽象基类"""
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):
    """第一个具体策略：为积分是1000或以上的顾客提供5%折扣。"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)


class BulkItemPromo(Promotion):
    """第二个具体策略：单个商品的数量为20个或以上时，提供10%折扣。"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount


class LargeOrderPromo(Promotion):
    """第三个具体策略：订单中不同商品的数量达到10个或以上时，提供7%折扣。"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [13]:
# Joe的积分为0
joe = Customer('John Doe', 0)
# Ann的积分为1100
ann = Customer('Ann Smith', 1100)
# 购物车中有3个商品
cart = (LineItem('banana', 4, Decimal('.5')), 
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5)))

In [14]:
# joe没有折扣
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [6]:
# Ann得到了5%折扣
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [15]:
# 有30个香蕉和10个苹果
banana_cart = (LineItem('banana', 30, Decimal('.5')),
               LineItem('apple', 10, Decimal('1.5')))
# joe购买时优惠了1.5美元
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [16]:
# 有10个不同的商品
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) 
                  for sku in range(10))

In [17]:
# 为joe提供了7%折扣
Order(joe, long_cart, LargeOrderPromo()) 

<Order total: 10.00 due: 9.30>

### 10.1.2 使用函数实现策略模式

把具体策略换成简单的函数，去掉抽象基类`Promotion`，Order类也修改了。

In [22]:
from dataclasses import dataclass
from typing import Callable

@dataclass(frozen=True)
class Order:
    customer: Customer
    cart: Sequence[LineItem]
    # 可以接收一个Order参数，并返回一个Decimal值的可调用对象
    promotion: Optional[Callable[['Order'], Decimal]] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'

def fidelity_promo(order: Order) -> Decimal:
    """第一个具体策略：为积分是1000或以上的顾客提供5%折扣。"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


def bulk_item_promo(order: Order) -> Decimal:
    """第二个具体策略：单个商品的数量为20个或以上时，提供10%折扣。"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


def large_order_promo(order: Order) -> Decimal:
    """第三个具体策略：订单中不同商品的数量达到10个或以上时，提供7%折扣。"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

### 10.1.3 选择最佳策略的简单方式

方法：直接计算所有策略，选取折扣幅度最大的那一个

In [23]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order: Order) -> Decimal:
    """选择可用的最佳折扣"""
    return max(promo(order) for promo in promos)

In [24]:
Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

In [25]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [26]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

### 10.1.4 找出一个模块中的全部策略

方法1：使用`globals`函数，帮助`best_promo`自动找到其他可用的`*_promo`函数。

In [27]:
promos = [promo for name, promo in globals().items()  
                if name.endswith('_promo') and        
                   name != 'best_promo'               
]


def best_promo(order: Order) -> Decimal:              
    """选择可用的最佳折扣"""
    return max(promo(order) for promo in promos)

方法2：使用`inspect.getmembers`函数获取对象的属性。

In [ ]:
import inspect
# 该模块包含所有的策略函数
import promotions

# 得到promotins模块下所有的函数对象
promos = [func for _, func in inspect.getmembers(promotions, inspect.isfunction)]


def best_promo(order: Order) -> Decimal:
    """选择可用的最佳折扣"""
    return max(promo(order) for promo in promos)

## 10.2 使用装饰器改进策略模式

In [32]:
Promotion = Callable[[Order], Decimal]

promos: list[Promotion] = []

# 注册装饰器
def promotion(promo: Promotion) -> Promotion:
    promos.append(promo)
    return promo


def best_promo(order: Order) -> Decimal:
    """选择可用的最佳折扣"""
    return max(promo(order) for promo in promos)  # <3>


@promotion
def fidelity(order: Order) -> Decimal:
    """第一个具体策略：为积分是1000或以上的顾客提供5%折扣。"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


@promotion
def bulk_item(order: Order) -> Decimal:
    """第二个具体策略：单个商品的数量为20个或以上时，提供10%折扣。"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


@promotion
def large_order(order: Order) -> Decimal:
    """第三个具体策略：订单中不同商品的数量达到10个或以上时，提供7%折扣。"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

优点：
1. 促销策略函数无须使用特殊的名称（不用以`_promo`结尾）。
2. `@promotion`装饰器突出了被装饰的函数作用，还便于临时禁用某个促销策略。
3. 促销折扣策略可以在其他模块中定义。